In [1]:
import pandas as pd
import os
import warnings; warnings.filterwarnings('ignore')

In [2]:
image_folder_path = "C:/Users/louis/github_vs/113_data/"

years = os.listdir(image_folder_path)
post_chat = years[5]
pre_chat = years[6]
years = years[0:5]
print(years)

['2017', '2018', '2019', '2020', '2021']


In [47]:
en_categories_df = pd.read_csv(image_folder_path + "CDS_translation_EN_NE.tsv", sep="\t")
en_categories_df["CDS (Nederlands)"] = en_categories_df["CDS (Nederlands)"].apply(lambda x: x.split('", "'))
s = en_categories_df.apply(lambda x: pd.Series(x["CDS (Nederlands)"]),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'CDS (NED)'
en_categories_df = en_categories_df.drop("CDS (Nederlands)", axis=1).join(s).reset_index(drop=True)
en_categories_df["CDS (NED)"] = en_categories_df["CDS (NED)"].apply( lambda x : x.replace('[',""))
en_categories_df["CDS (NED)"] = en_categories_df["CDS (NED)"].apply( lambda x : x.replace(']',""))
en_categories_df["CDS (NED)"] = en_categories_df["CDS (NED)"].apply( lambda x : x.replace('"',""))
en_categories_df["CDS (NED)"] = en_categories_df["CDS (NED)"].str.strip()

def translate_cds(cds,disp =False):
    en_cds = []
    ne_cds = []
    for c in cds:
        trans = en_categories_df[en_categories_df['CDS (NED)']==c]
        if len(trans) ==0: 
            if disp: print(c)
            continue
        trans_str = str(trans['CDS (English)'].values).replace('[', " ")
        trans_str = trans_str.replace(']', " ")
        trans_str = trans_str.replace("' '", "/")
        trans_str = trans_str.replace("'", "").strip()       
        en_cds.append(trans_str)
        ne_cds.append(c)
    print("Number of translated columns: ",len(en_cds)," out of ",len(cds))
    return en_cds,ne_cds

def categorise(df_all,disp =False):
    for category in en_categories_df['categories'].unique():
        category_cols = en_categories_df[en_categories_df['categories']== category]['CDS (NED)'].str.strip().to_list()
        common_cols = list(set(category_cols).intersection(set(df_all.columns)))
        if disp: print("category =",category,"common_cols =",len(common_cols))
        df_all[category] = df_all[common_cols].sum(axis=1)
    return df_all
  
def translate(df_all):      
    cols = df_all.columns.tolist()
    cols_EN,cols_NE = translate_cds(cols)
    df_all.rename(columns = dict(zip(cols_NE,cols_EN)) , inplace=True)
    return df_all
en_categories_df


,categories,CDS (English),CDS (NED)
0,Labeling and mislabeling,I am a,ik ben een
1,Labeling and mislabeling,he is a,hij is een
2,Labeling and mislabeling,she is a,z(e|ij) is een
3,Labeling and mislabeling,they are a,z(e|ij) zijn een
4,Labeling and mislabeling,it is a,(het|'t) is een
...,...,...,...
307,Should statements,must,moet(en)?
308,Should statements,have to,moeten
309,Should statements,have to,heb(ben)? moeten
310,Should statements,have to,had(den)? moeten


In [25]:
en_categories_df['categories'].unique()

array(['Labeling and mislabeling', 'Catastrophizing',
       'Dichotomous Reasoning', 'Emotional Reasoning',
       'Disqualifying the Positive', 'Magnification and Minimization',
       'Mental Filtering', 'Mindreading', 'Fortune-telling',
       'Overgeneralizing', 'Personalizing', 'Should statements'],
      dtype=object)

In [49]:
# retrive the message information data
def get_msg_info_data():
    df_all = pd.DataFrame(columns =['event_id', 'message_id', 'user_handle', 'sec_since_start']);
    for y in years:
        
        events =  os.listdir(image_folder_path+y)

        for e in events:
            df = pd.read_csv(image_folder_path+y+'/'+e+'/'+"message_info.tsv",sep= '\t')
            df_all = df_all.merge(df, how = "outer")
            
# remove all 'system' messages
    df_all.drop(df_all[df_all['user_handle'].str.contains('system')].index, inplace = True)

    df_all["id"] = df_all["event_id"].astype("string") + df_all["message_id"].astype("string")
    df_all.drop(['message_id'],axis=1,inplace=True)
    df_all=df_all.set_index("event_id",drop=False)
    # replacing all the unique employees id to a general employee user
    df_all['user_handle'] = df_all['user_handle'].replace("employee.*","employee",regex = True)
    return df_all

# combination with chat dataset
def combine_with_chatquestions_data(df_all,df_quest):
    df_quest = df_quest.set_index("event_id")
    df_all = df_quest.join(df_all,how='inner')

    df_all.set_index('id',inplace=True)
    # transforming the chat question into positive and negative states
    pos = df_all['Ik heb de wil om te leven']
    neg =  df_all.drop(['Ik heb de wil om te leven','user_handle','sec_since_start'],axis=1).mean(axis=1)
    df_all = pd.DataFrame({"postive":pos,"negative":neg,'user_handle':df_all['user_handle'],'sec_since_start':df_all['sec_since_start'],'event_id':df_all['event_id']})

    return df_all

# include cds data in the Dataframe
def combine_with_category_data(df_all,years):
    
    categories_names = en_categories_df['categories'].unique()
    for col in categories_names:
            df_all[col] = 0

    all_common_indexes = []
    for y in years:
        print("year =",y)
        events =  os.listdir(image_folder_path+y)

        for e in events:
            
            df = pd.read_csv(image_folder_path+y+'/'+e+'/'+"CDS_results.tsv",sep= '\t')
            df.drop(common_cds,axis=1,inplace=True)
            
            df['id'] = df['event_id'].astype("string") + df['message_id'].astype("string")
            df.set_index("id",inplace=True)
            df.drop(['message_id'],axis=1,inplace=True)
            
            df_common_indexes = df.index.intersection(df_all.index)
            if len(df_common_indexes) == 0: print(str(0),'--'); continue #it is observed that years 2018 and 2019 did not have chat questions
            all_common_indexes.extend(df_common_indexes)
            df = categorise(df)
            
            df_all.loc[df_common_indexes,categories_names] = df.loc[df_common_indexes,categories_names]
        
            
        print("-------------------------------------------------------")
        
    print("all_common_indexes =",len(all_common_indexes))
    print("all_indexes =",len(df_all.index))
    non_common_indexes = list(set(df_all.index) - set(all_common_indexes))
    df_all.drop(non_common_indexes,inplace=True)
    
    return df_all
     
def combine_with_cds_data(df_all,years):
    
    df = pd.read_csv(image_folder_path+'/2021'+'/'+"202101"+'/'+"CDS_results.tsv",sep= '\t')
    cds_names =df.columns[2:].to_list()
    # removing cds that have less then 50 occurences in total over all the conversations (extracted from EDA phase3)
    common_cds = ['hij zal( .+ | )niet denken', 'w(ij|e) zullen( .+ | )weten', 'een onbekwa(am|me)', 'w(ij|e) zullen( .+ | )niet geloven', 'een niet geliefd', 'zwart-/wit', 'w(ij|e) zullen( .+ | )niet weten', 'w(ij|e) zullen( .+ | )niet denken', 'zal( .+ | )verkeerd gaan', 'j(ij|e) (zal|zult)( .+ | )niet weten', 'j(ij|e) (zal|zult)( .+ | )niet denken', 'zal onmogelijk zijn', '(zal|gaat) niet plaatsvinden', 'wordt vreselijk', 'zal vreselijk zijn', 'wordt verschrikkelijk', 'zal verschrikkelijk zijn', 'wordt een ramp', 'zal( .+ | )een ramp zijn', '(zal|gaat) nooit eindigen', 'we zullen( .+ | ) niet', 'j(ij|e) (zal|zult)( .+ | ) niet', 'w(ij|e) zullen( .+ | )geloven', 'z(ij|e) zullen( .+ | )niet denken', 'z(ij|e) zullen( .+ | ) niet', 'afdoende,? maar', 'iedereen zal( .+ | )geloven', 'hij zal( .+ | )niet geloven', 'hij zal( .+ | )geloven', 'ik zie slechts', 'hij zal( .+ | )niet weten', 'zal( .+ | )mis gaan', 'eindigt niet', 'acceptabel,? maar', 'z(ij|e) zal( .+ | )niet denken', 'alleen het (aller)?ergste', 'alleen het slechtste', 'w(ij|e) zullen( .+ | )denken', 'j(ij|e) (zal|zult)( .+ | )niet geloven', 'ik veroorzaakte', 'z(ij|e) zal( .+ | )niet weten', 'wordt onmogelijk', '(zal|gaat) niet ophouden', 'j(ij|e) (zal|zult)( .+ | )geloven', 'ik zal( .+ | ) niet', 'een incompetente?', 'z(ij|e) zullen( .+ | )niet weten', 'z(ij|e) zal( .+ | )niet geloven', 'iedereen zal( .+ | )weten', 'een beschadigde?', 'zal( .+ | )fout gaan', 'iedereen zal( .+ | )denken', '(zal|gaat) niet stoppen', 'waardeloos in', 'een volslagen', 'zwart of wit', 'geef mij de schuld', 'z(ij|e) zullen( .+ | )niet geloven', 'z(ij|e) zal( .+ | )geloven', 'een toxische?', 'hij zal( .+ | )weten', '(zal|gaat) nooit stoppen', 'alleen het (slechte|verkeerde)', 'een ongewenste?', 'niemand zal( .+ | )denken', 'niemand zal( .+ | )weten', 'j(ij|e) (zal|zult)( .+ | )weten', 'een giftige?', 'z(ij|e) zullen( .+ | )geloven', 'iedereen gelooft', 'w(ij|e) geloven( .+ | )niet', 'een hulpelo(os|ze)', 'z(ij|e) zal( .+ | )weten', 'het enige (w|d)at ik zie', 'ik ben verantwoordelijk', 'niemand zal( .+ | )geloven', 'hij zal( .+ | )denken', 'een absolu(ut|te)', 'z(ij|e) zal( .+ | )denken', 'irrelevant', 'een afschuwelijke?', 'gebeurt (elke|iedere) keer', 'mij de schuld geven', 'w(ij|e) geloven', 'niet echt top', 'niet echt geweldig', 'z(ij|e) zullen( .+ | )weten', '(een )?ongeliefde?', 'een kapotte', '(he|al)lemaal door mij', 'top,? maar', 'voelt nog steeds', 'telt niet', 'niemand denkt', 'gaat wel,? maar', 'ik voel m(ij|e) verantwoordelijk', 'hij gelooft( .+ | )niet', 'z(ij|e) zullen( .+ | )denken', 'gebeurt altijd', 'geweldig,? maar', 'een lelijke?', 'z(e|ij) zijn een']
    cds_col_remained = []
    for col in cds_names:
        if col not in common_cds: 
            cds_col_remained.append(col)
            df_all[col] = 0

    all_common_indexes = []
    for y in years:
        print("year =",y)
        events =  os.listdir(image_folder_path+y)

        for e in events:
            
            df = pd.read_csv(image_folder_path+y+'/'+e+'/'+"CDS_results.tsv",sep= '\t')
            df.drop(common_cds,axis=1,inplace=True)
            
            df['id'] = df['event_id'].astype("string") + df['message_id'].astype("string")
            df.set_index("id",inplace=True)
            df.drop(['message_id'],axis=1,inplace=True)
            
            df_common_indexes = df.index.intersection(df_all.index)
            if len(df_common_indexes) == 0: continue #it is observed that years 2018 and 2019 did not have chat questions
            all_common_indexes.extend(df_common_indexes)
            
            df_all.loc[df_common_indexes,cds_col_remained] = df.loc[df_common_indexes,cds_col_remained]
        
            
        print("-------------------------------------------------------")
        
    print("all_common_indexes =",len(all_common_indexes))
    print("all_indexes =",len(df_all.index))
    non_common_indexes = list(set(df_all.index) - set(all_common_indexes))
    df_all.drop(non_common_indexes,inplace=True)
    
    return df_all           

In [51]:
df_all = get_msg_info_data()
df_pre = pd.read_csv(image_folder_path+"prechat_questions.tsv",sep= '\t').dropna()
df_all = combine_with_chatquestions_data(df_all,df_pre)
df_all = combine_with_cds_data(df_all,years)
df_all = categorise(df_all)
df_all = translate(df_all)

year = 2017
-------------------------------------------------------
year = 2018
-------------------------------------------------------
year = 2019
-------------------------------------------------------
year = 2020
-------------------------------------------------------
year = 2021
-------------------------------------------------------
all_common_indexes = 2075242
all_indexes = 2075242
Number of translated columns:  163  out of  180


In [53]:
df_all.to_csv("df_all_prechat.tsv",sep='\t')

In [54]:
# extracting a dataset with the postchat questions, using the same pipeling as the above
df_all = get_msg_info_data()
df_post = pd.read_csv(image_folder_path+"postchat_questions.tsv",sep= '\t').dropna()
df_all = combine_with_chatquestions_data(df_all,df_post)
df_all = combine_with_cds_data(df_all,years)
df_all = categorise(df_all)
df_all = translate(df_all)

year = 2017
-------------------------------------------------------
year = 2018
-------------------------------------------------------
year = 2019
-------------------------------------------------------
year = 2020
-------------------------------------------------------
year = 2021
-------------------------------------------------------
all_common_indexes = 371753
all_indexes = 371753
Number of translated columns:  163  out of  180


In [55]:
df_all.to_csv("df_all_postchat.tsv",sep='\t')

In [57]:
df_all = get_msg_info_data()
df_all.set_index("id",inplace=True)
df_all = combine_with_category_data(df_all,years)
df_all

year = 2017
-------------------------------------------------------
year = 2018
-------------------------------------------------------
year = 2019
-------------------------------------------------------
year = 2020
-------------------------------------------------------
year = 2021
-------------------------------------------------------
all_common_indexes = 6350654
all_indexes = 6350654


,event_id,user_handle,sec_since_start,Labeling and mislabeling,Catastrophizing,Dichotomous Reasoning,Emotional Reasoning,Disqualifying the Positive,Magnification and Minimization,Mental Filtering,Mindreading,Fortune-telling,Overgeneralizing,Personalizing,Should statements
id,,,,,,,,,,,,,,,
cl0709091,cl070909,employee,9.0,0,0,1,0,0,0,0,0,0,0,0,0
cl0709092,cl070909,client,44.0,0,0,0,0,0,0,0,0,0,0,0,0
cl0709093,cl070909,employee,64.0,0,0,0,0,0,0,0,0,0,0,0,0
cl0709096,cl070909,employee,88.0,0,0,0,0,0,0,0,0,0,0,0,0
cl0709098,cl070909,client,222.0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cl002068113,cl002068,employee,4649.0,0,0,0,0,0,0,0,0,0,0,0,0
cl002068114,cl002068,employee,4812.0,0,0,0,0,0,0,0,0,0,0,0,0
cl0020694,cl002069,employee,25.0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
df_all.to_csv("df_all_nochat.tsv",sep='\t')